<a href="https://colab.research.google.com/github/ArifAygun/Webscraping/blob/main/AA_Global_Companies__Ranked_by_Revenue_2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Web_Scraping/Companiesmarketcap/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Web_Scraping/Companiesmarketcap


In [45]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [46]:
response = requests.get(base_site1)
html = response.content
soup = BeautifulSoup(html, "html.parser")

In [47]:
def pageCountFound():
    base_site1 = f"https://companiesmarketcap.com/largest-companies-by-revenue/"
    response = requests.get(base_site1)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    rowCount = int(soup.find('span', {'class':'companies-count'}).text.replace(',', ''))
    pageCount = (rowCount + 99) // 100  # Calculate pageCount based on total rows
    return pageCount

pageCountFound()

80

In [62]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def pageCountFound():
    base_site1 = f"https://companiesmarketcap.com/largest-companies-by-revenue/"
    response = requests.get(base_site1)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    rowCount = int(soup.find('span', {'class':'companies-count'}).text.replace(',', ''))
    pageCount = (rowCount + 99) // 100  # Calculate pageCount based on total rows
    return pageCount

def finalPulling():
    df_out = pd.DataFrame(columns=["Rank", "Company Name", "Company Link", "Market Cap", "Price", "Percentage Change", "Country"])
    pageCount = pageCountFound()

    for pageNo in range(1, pageCount + 1):
        base_site = f"https://companiesmarketcap.com/largest-companies-by-revenue/?page={pageNo}"
        response = requests.get(base_site)
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        tbody = soup.find('tbody')

        ranks_list = tbody.find_all('td', {'class':'rank-td td-right'})
        ranks = [i.text.strip() for i in ranks_list]

        name_list = tbody.find_all('td', {'class':'name-td'})
        names = []
        name_links = []

        for name_td in name_list:
            company_name_elem = name_td.select_one('div.company-name a')
            if company_name_elem:
                names.append(company_name_elem.text.strip())
                name_links.append(company_name_elem.get("href"))
            else:
                names.append("")
                name_links.append("")

        cap_links = tbody.find_all('td', {'class':'td-right'})
        caps_links_list = [i.text.strip() for i in cap_links]

        prices = [caps_links_list[i] for i in range(1, len(caps_links_list), 3)]
        percents = [caps_links_list[i] for i in range(2, len(caps_links_list), 3)]
        caps = [caps_links_list[i] for i in range(0, len(caps_links_list), 3)]

        country_links = tbody.find_all('span', {'class':'responsive-hidden'})
        countries = [i.text.strip() for i in country_links]

        df_in = pd.DataFrame({
            "Rank": ranks,
            "Company Name": names,
            "Company Link": name_links,
            "Market Cap": caps,
            "Price": prices,
            "Percentage Change": percents,
            "Country": countries
        })

        df_out = pd.concat([df_out, df_in], ignore_index=True)

    df_out.to_csv("df_out.csv", index=False)
    return df_out

finalPulling()


,Rank,Company Name,Company Link,Market Cap,Price,Percentage Change,Country
0,1,,,1,$1.184 T,$211.35,Russia
1,2,,,2,$630.79 B,$163.19,USA
2,3,,,3,$538.04 B,$139.06,USA
3,4,,,4,$535.54 B,$9.15,S. Arabia
4,5,,,5,$469.29 B,$0.84,China
...,...,...,...,...,...,...,...
7956,7957,,,7957,-$7.65 B,$2.53,UK
7957,7958,,,7958,-$14.42 B,$10.48,UK
7958,7959,,,7959,-$25.25 B,$4.96,UK
7959,7960,,,7960,-$39.21 B,$6.66,UK
